# First Login

In [1]:
# syft absolute
import syft as sy

In [2]:
node = sy.orchestra.launch(
    name="test-domain-1",
    port="auto",
    dev_mode=True,
    reset=True,
    n_consumers=1,
    create_producer=True,
)
domain_client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Starting test-domain-1 server on 0.0.0.0:19885
Waiting for server to start.

INFO:     Started server process [97779]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:19885 (Press CTRL+C to quit)


.
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=4c9dc528c8774fa7bf90259b3d710215 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
INFO:     127.0.0.1:54127 - "GET /api/v2/metadata HTTP/1.1" 200 OK
 Done.
INFO:     127.0.0.1:54129 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:54129 - "GET /api/v2/metadata HTTP/1.1" 200 OK
Logged into <test-domain-1: High-side Domain> as GUEST
INFO:     127.0.0.1:54129 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:54129 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:54131 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# Modify Admin User/Password

In [3]:
# TODO

# Create Adapter to Vertex

In [4]:
PRIVATE_KEY = ""

In [5]:
SERVICE_ACCOUNT = {
    "type": "service_account",
    "project_id": "project-enigma-415021",
    "private_key_id": "0bd7cdd831f456f905fa98ad570740948bf7b7b9",
    "private_key": PRIVATE_KEY,
    "client_email": "vertex-test@project-enigma-415021.iam.gserviceaccount.com",
    "client_id": "113559790781665979367",
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/vertex-test%40project-enigma-415021.iam.gserviceaccount.com",
    "universe_domain": "googleapis.com",
}

In [7]:
@sy.api_endpoint(
    path="vertex.run",
    settings={"SERVICE_ACCOUNT": SERVICE_ACCOUNT},
)
def public_endpoint_method(
    context,
    prompt: str,
    max_tokens: int = 50,
    temperature: float = 0.1,
    top_p: float = 1.0,
    top_k: int = 1,
    raw_response: bool = False,
) -> str:
    # context.state = ["jimmy@caltech.edu": {"n_calls":3, "period_secs":60}]

    # use helper function rate limiter
    # is_allowed = allowed(context.state, context.user.email)

    # third party
    from google.cloud import aiplatform
    from google.oauth2 import service_account

    try:
        credentials = service_account.Credentials.from_service_account_info(
            context.settings["SERVICE_ACCOUNT"]
        )

        PROJECT_ID = "project-enigma-415021"
        REGION = "us-west1"
        ENDPOINT_ID = "3213239169291649024"
        aip_endpoint_name = (
            f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}"
        )
        endpoint_vllm = aiplatform.Endpoint(aip_endpoint_name, credentials=credentials)
        default_kwargs = {
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "top_k": top_k,
            "raw_response": raw_response,
        }
        instances = [
            default_kwargs,
        ]
        response = endpoint_vllm.predict(instances=instances)
        prediction = response.predictions[0]
    except Exception as e:
        prediction = f"Error: Please try again? {e}"
    return {"prediction": prediction}

In [8]:
# use to delete if you want to add again (there is a seperate update API)
domain_client.api.services.api.delete(endpoint_path="vertex.run")

INFO:     127.0.0.1:54134 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftError: Endpoint path: vertex.run does not exist.

In [9]:
domain_client.api.services.api.add(endpoint=public_endpoint_method)

INFO:     127.0.0.1:54136 - "POST /api/v2/api_call HTTP/1.1" 200 OK


SyftSuccess: Endpoint successfully created.

In [ ]:
# test

In [10]:
domain_client.refresh()

INFO:     127.0.0.1:54138 - "GET /api/v2/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f&communication_protocol=dev HTTP/1.1" 200 OK


In [19]:
domain_client.api.services.vertex.run(prompt="What color is the sky?")

INFO:     127.0.0.1:54219 - "POST /api/v2/api_call HTTP/1.1" 200 OK


{'prediction': 'Prompt:\nWhat color is the sky?\nOutput:\nWhat color is the sky?\n\nWhat color is the sky?\n\nWhat color is the sky?\n\nWhat color is the sky?\n\nWhat color is the sky?\n\nWhat color is the sky?\n\nWhat color is the sky?'}

Logged into <test-domain-1: High side Domain > as GUEST


# Create Rate Limit Updater

In [ ]:
# TODO

In [20]:
# Cleanup local domain server
if node.node_type.value == "python":
    node.land()